I have a Twitter account that I have not used in quite awhile. I've wanted to write a chatbot that would do some posting for me and finally have produced the script below, taken nearly verbatim from twiboozer's GitHub documentation, it generates Markov Chains of text based off your timeline. 

<blockquote class="twitter-tweet" lang="en"><p lang="en" dir="ltr">Been finding the time to tweet to hard, will let my friend Markov take over for awhile. ;-)</p>&mdash; Rob Berkes (@KosmikDebris) <a href="https://twitter.com/KosmikDebris/status/626544259311648770">July 30, 2015</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

(thanks https://github.com/romansalin/twiboozer)

In [ ]:
from twiboozer import TwiBot, MarkovChain


OAUTH_TOKEN = ''
OAUTH_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
twibot = TwiBot(oauth_token=OAUTH_TOKEN, oauth_secret=OAUTH_SECRET, 
                consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET)
tweets = twibot.get_timeline(count=500, include_rts=True, include_replies=False)

mc = MarkovChain()
mc.generate_database(tweets.decode())
tweet = mc.generate_string()


twibot.post_tweet(tweet)

Just fill in Token, Keys, and Secrets with info from an App you create in your Twitter accounts Developer page (make sure permissions are read AND write), and you're set to go.  

After handing over posting privileges, it's very first tweet to my timeline was, well a bit creepy I have to admit ... 

<blockquote class="twitter-tweet" lang="en"><p lang="en" dir="ltr">thanks.</p>&mdash; Rob Berkes (@KosmikDebris) <a href="https://twitter.com/KosmikDebris/status/626545937343578112">July 30, 2015</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

More typical, of course are tweets like these ...

<blockquote class="twitter-tweet" lang="en"><p lang="en" dir="ltr">Amnesty International (AI) receives suppo from Garett I&#39;d love Mordin :).</p>&mdash; Rob Berkes (@KosmikDebris) <a href="https://twitter.com/KosmikDebris/status/626770639081189376">July 30, 2015</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

Now, it sounds a bit nonsensical at this point. I'd like to work on optimizing the Markov chain, see if I can use other Python Markov Chain modules with twiboozer to produce better results, for example.  For example, I'd like it to sound more like me, of course. To accomplish this, I'd need the sample texts, the corpus, to be samples of my own writing, say my own Twitter or Facebook feed, instead of the feed of other users.

Thankfully, both Twitter and Facebook now make downloading your own archive of posts fairly easy.  For parsing out your own messages and saving them into a text file, I used the BeautifulSoup4 web scraping library...

In [ ]:
#This script creates a corpus, or word dictionary, from my Facebook wall of posts.  It can then be used to build as
#needed the bigrams, trigams, and sentences for a Markov chain
from bs4 import BeautifulSoup
ifp=open('/home/rob/Dropbox/Data/html/wall.htm','r')
soup=BeautifulSoup(ifp,"lxml")
comments=soup.find_all("div",class_="comment")
ifp.close()
ofp=open('/home/rob/Dropbox/Data/corpus.facebook','w')
for comment in comments:
    ofp.write(comment.string)
ofp.close()

The Markovify library will allow us to create Markov chains using file pointers, allowing us to use our Facebook corpus as below..

In [ ]:
#now see https://github.com/jsvine/markovify
import markovify

# Get raw text as string.
with open("/home/rob/Dropbox/Data/corpus.facebook") as f:
    text = f.read()
    
# Build the model.
text_model = markovify.Text(text)

# Print three randomly-generated sentences of no more than 140 characters
i=0
while i<3:
    NEWSENT=text_model.make_short_sentence(140)
    if NEWSENT != "NONE":
        i+=1
        print(NEWSENT)

Putting this all together, we have a script that combines our own word style with the last 500 tweets in our timeline to create sentences  based both on our own style and the latest events in our newsfeed!

In [ ]:
import markovify
from twiboozer import TwiBot


OAUTH_TOKEN = ''
OAUTH_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

twibot = TwiBot(oauth_token=OAUTH_TOKEN, oauth_secret=OAUTH_SECRET, 
                consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET)
tweets = twibot.get_timeline(count=500, include_rts=True, include_replies=False)

TWEETS=tweets.decode()

with open("/home/rob/Dropbox/Data/corpus.facebook") as f:
    text = f.read()
    
text+=TWEETS
text_model = markovify.Text(text)

NEWSENT=text_model.make_short_sentence(140)
twibot.post_tweet(NEWSENT)

Running our new script through both Facebook and Twitter now, our Tweets now look a bit more like complete sentences with an intriquing amount of structure. 

<blockquote class="twitter-tweet" lang="en"><p lang="en" dir="ltr">And lucky me, one of my head....McDonald&#39;s tells worker she can ONLY be paid with a narcoleptic pilot.</p>&mdash; Rob Berkes (@KosmikDebris) <a href="https://twitter.com/KosmikDebris/status/626885540952932352">July 30, 2015</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

or
<blockquote class="twitter-tweet" lang="en"><p lang="en" dir="ltr">Visual Basic You’ll shoot yourself in the appendage which holds the gun into the Amazon App store!</p>&mdash; Rob Berkes (@KosmikDebris) <a href="https://twitter.com/KosmikDebris/status/626885105257070597">July 30, 2015</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>